# Бэггинг и случайный лес

In [146]:
from sklearn import datasets, tree, cross_validation, ensemble
import numpy as np

In [12]:
# генерируем датасет
digits = datasets.load_digits()

In [13]:
# разбивка данных на признаки и целевые метки
data = digits.data
target = digits.target

### 1. Решающее дерево

In [52]:
# строим модель Решающее Дерево с настройками по умолчанию
DT_classifier = tree.DecisionTreeClassifier() 

In [15]:
# оценм качество леса на наших данных посредством кросс-валидации
scoring  = cross_validation.cross_val_score(DT_classifier, data, target, cv = 10)

In [16]:
scoring

array([ 0.78918919,  0.85245902,  0.83977901,  0.83333333,  0.78212291,
        0.88826816,  0.88268156,  0.8258427 ,  0.85310734,  0.81818182])

In [17]:
print 'Среднее качество модели: {},    Отклонение: {}'.format(scoring.mean(), scoring.std())

Среднее качество модели: 0.836496502958,    Отклонение: 0.0331787095981


In [18]:
# запишем первый ответ (качество решающего дерева) в файл
with open("answer_1.txt", "w") as F:
    F.write(str(scoring.mean()))

### 2. Беггинг

In [22]:
# строим модель Беггинг с 50 деревьями по модели DT_classifier, остальные настройки по умолчанию
B_classifier = ensemble.BaggingClassifier(DT_classifier, n_estimators = 50)

In [24]:
# оценм качество Беггинга на наших данных посредством кросс-валидации
scoring  = cross_validation.cross_val_score(B_classifier, data, target, cv = 10)

In [25]:
scoring

array([ 0.87567568,  0.95081967,  0.92265193,  0.95      ,  0.91620112,
        0.98882682,  0.94972067,  0.91573034,  0.87570621,  0.90340909])

In [26]:
print 'Среднее качество модели: {},    Отклонение: {}'.format(scoring.mean(), scoring.std())

Среднее качество модели: 0.924874152754,    Отклонение: 0.0338736845227


In [27]:
# запишем второй ответ (качество Беггинга) в файл
with open("answer_2.txt", "w") as F:
    F.write(str(scoring.mean()))

### 3. Беггинг c количеством случаных признаков для каждого дерева = √d

In [85]:
# находим процентное отношение корня признаков ко всем признакам
sqrt_d = np.sqrt(data[0].size) / data[0].size

# строим модель Беггинг для решающего дерева с количеством признаков равным корню из всех признаков
B_classifier = ensemble.BaggingClassifier(DT_classifier, n_estimators = 50, max_features = sqrt_d)

In [86]:
# оценм качество Беггинга на наших данных посредством кросс-валидации
scoring  = cross_validation.cross_val_score(B_classifier, data, target, cv = 10)

In [87]:
scoring

array([ 0.8972973 ,  0.95081967,  0.92265193,  0.9       ,  0.91620112,
        0.91620112,  0.93854749,  0.97752809,  0.89265537,  0.91477273])

In [88]:
print 'Среднее качество модели: {},    Отклонение: {}'.format(scoring.mean(), scoring.std())

Среднее качество модели: 0.922667480819,    Отклонение: 0.0250170937928


In [89]:
# запишем третий ответ (качество Беггинга 2) в файл
with open("answer_3.txt", "w") as F:
    F.write(str(scoring.mean()))

### 4. Беггинг c количеством случаных признаков для каждой вершины каждого дерева = √d 
На самом деле данный алгоритм в точности повторит Случайный Лес

In [47]:
# строим модель Решающее Дерево с количеством признаков для каждой вершины равным корню из всех признаков
DT_classifier = tree.DecisionTreeClassifier(max_features = "sqrt")

# строим модель Беггинг для данного решающего дерева
B_classifier = ensemble.BaggingClassifier(DT_classifier, n_estimators = 50)

In [48]:
# оценм качество Беггинга на наших данных посредством кросс-валидации
scoring  = cross_validation.cross_val_score(B_classifier, data, target, cv = 10)

In [49]:
scoring

array([ 0.92432432,  0.97814208,  0.9281768 ,  0.96111111,  0.94972067,
        0.97206704,  0.98324022,  0.96629213,  0.93220339,  0.94318182])

In [50]:
print 'Среднее качество модели: {},    Отклонение: {}'.format(scoring.mean(), scoring.std())

Среднее качество модели: 0.953845958332,    Отклонение: 0.0203066421835


In [51]:
# запишем четвертый ответ (качество Беггинга 3) в файл
with open("answer_4.txt", "w") as F:
    F.write(str(scoring.mean()))

### 5. Построение случайного леса и ответы на вопросы

In [145]:
# строим случайный лес из 50 деревьев
RF_classifier = ensemble.RandomForestClassifier(n_estimators = 50, max_features = "sqrt")
# оценм качество Случайного Леса на наших данных посредством кросс-валидации
scoring  = cross_validation.cross_val_score(RF_classifier, data, target, cv = 10)
print 'Среднее качество модели: {},    Отклонение: {}'.format(scoring.mean(), scoring.std())

Среднее качество модели: 0.95391188317,    Отклонение: 0.0240434532513


1) Случайный лес сильно переобучается с ростом количества деревьев 
- Нет

2) При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев
- Да

3) С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.
- Да

4) При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.
- Да

5) При большом количестве признаков (40, 50, 60) качество классификации лучше, чем при малом количестве признаков (5, 10). Это связано с тем, что чем больше признаков - тем больше информации об объектах, а значит алгоритм может делать прогнозы более точно.
- Нет

6) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса намного лучше, чем без ограничения глубины, т.к. деревья получаются не переобученными. С ростом глубины деревьев качество ухудшается.
- Нет

7) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).
- Да

In [139]:
# запишем пятый ответ в файл
with open("answer_5.txt", "w") as F:
    F.write("2 3 4 7")